In [1]:
# CSC 478 - Programming Data Mining Applications - Professor Bamshad Mobasher
# Student: Ricardo Barros Lourenço
# Final Project Notebook

In [2]:
import numpy as np
import pandas as pd
from osgeo import gdal, ogr, osr
from sklearn import decomposition, preprocessing, linear_model, cross_validation
from sklearn.metrics import pairwise, classification_report, mean_squared_error
import sys
import gc
import cPickle
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Functions

# Loads non stacked GeoTiff 
def load_image(image_path):
    ds = gdal.Open( path )
    myarray = np.array(ds.GetRasterBand(1).ReadAsArray())
    image_rows, image_columns = myarray.shape
    image_vector = myarray.reshape((image_rows*image_columns),1)
    return (image_vector, image_rows, image_columns)

def extract_factors(bands_matrix, components):
    pca = decomposition.PCA(n_components=components, whiten=False)
    factors = pca.fit_transform(bands_matrix)
    print 'Evaluation of results using Principal Component Analysis(PCA) as data reduction criteria'
    print 'Explained variance using ',components,' components: ',(sum(pca.explained_variance_ratio_))*100,'%'
    return factors

def normalize(vector):
    # MinMax Normalization for a vector
    min_max_scaler = preprocessing.MinMaxScaler()
    normalized = min_max_scaler.fit_transform(np.array(vector))
    return normalized

# NDVI Calculation
ndvi = lambda NIR, RED: (NIR - RED)/(NIR - RED)

In [4]:
# Load B1
path = 'https://s3-us-west-2.amazonaws.com/landsat-pds/L8/221/071/LC82210712014261LGN00/LC82210712014261LGN00_B1.TIF'
B1_vector, B1_rows, B1_collumns = load_image(path)

In [5]:
# Load B2
path = 'https://s3-us-west-2.amazonaws.com/landsat-pds/L8/221/071/LC82210712014261LGN00/LC82210712014261LGN00_B2.TIF'
B2_vector, B2_rows, B2_collumns = load_image(path)

In [6]:
# Load B3
path = 'https://s3-us-west-2.amazonaws.com/landsat-pds/L8/221/071/LC82210712014261LGN00/LC82210712014261LGN00_B3.TIF'
B3_vector, B3_rows, B3_collumns = load_image(path)

In [7]:
# Load B4
path = 'https://s3-us-west-2.amazonaws.com/landsat-pds/L8/221/071/LC82210712014261LGN00/LC82210712014261LGN00_B4.TIF'
B4_vector, B4_rows, B4_collumns = load_image(path)

In [8]:
# Load B5
path = 'https://s3-us-west-2.amazonaws.com/landsat-pds/L8/221/071/LC82210712014261LGN00/LC82210712014261LGN00_B5.TIF'
B5_vector, B5_rows, B5_collumns = load_image(path)

In [9]:
# Load B6
path = 'https://s3-us-west-2.amazonaws.com/landsat-pds/L8/221/071/LC82210712014261LGN00/LC82210712014261LGN00_B6.TIF'
B6_vector, B6_rows, B6_collumns = load_image(path)

In [10]:
## Load B7
path = 'https://s3-us-west-2.amazonaws.com/landsat-pds/L8/221/071/LC82210712014261LGN00/LC82210712014261LGN00_B7.TIF'
B7_vector, B7_rows, B7_collumns = load_image(path)

In [11]:
# Load B9
path = 'https://s3-us-west-2.amazonaws.com/landsat-pds/L8/221/071/LC82210712014261LGN00/LC82210712014261LGN00_B9.TIF'
B9_vector, B9_rows, B9_collumns = load_image(path)

In [12]:
# Load B10
path = 'https://s3-us-west-2.amazonaws.com/landsat-pds/L8/221/071/LC82210712014261LGN00/LC82210712014261LGN00_B10.TIF'
B10_vector, B10_rows, B10_collumns = load_image(path)

In [13]:
print 'B1: ',B1_vector.mean(),'(mean)',B1_vector.min(),'(min)',B1_vector.max(),'(max)'
print 'B2: ',B2_vector.mean(),'(mean)',B2_vector.min(),'(min)',B2_vector.max(),'(max)'
print 'B3: ',B3_vector.mean(),'(mean)',B3_vector.min(),'(min)',B3_vector.max(),'(max)'
print 'B4: ',B4_vector.mean(),'(mean)',B4_vector.min(),'(min)',B4_vector.max(),'(max)'
print 'B5: ',B5_vector.mean(),'(mean)',B5_vector.min(),'(min)',B5_vector.max(),'(max)'
print 'B6: ',B6_vector.mean(),'(mean)',B6_vector.min(),'(min)',B6_vector.max(),'(max)'
print 'B7: ',B7_vector.mean(),'(mean)',B7_vector.min(),'(min)',B7_vector.max(),'(max)'
print 'B9: ',B9_vector.mean(),'(mean)',B9_vector.min(),'(min)',B9_vector.max(),'(max)'
print 'B10: ',B10_vector.mean(),'(mean)',B10_vector.min(),'(min)',B10_vector.max(),'(max)'

B1:  7353.14671323 (mean) 0 (min) 46458 (max)
B2:  6966.16455043 (mean) 0 (min) 31296 (max)
B3:  6945.18293475 (mean) 0 (min) 33367 (max)
B4:  7460.81384237 (mean) 0 (min) 41116 (max)
B5:  11338.7542367 (mean) 0 (min) 47920 (max)
B6:  12296.4527858 (mean) 0 (min) 65535 (max)
B7:  8983.41003754 (mean) 0 (min) 63533 (max)
B9:  3617.888978 (mean) 0 (min) 13440 (max)
B10:  22031.4595995 (mean) 0 (min) 65535 (max)


In [14]:
# PCA Analysis - Band 8 not used because of characteristics 
bands_matrix = np.concatenate((normalize(B1_vector), normalize(B2_vector), normalize(B3_vector), normalize(B4_vector), normalize(B5_vector), normalize(B6_vector), normalize(B7_vector), normalize(B9_vector), normalize(B10_vector)), axis=1)
original_input_matrix = np.concatenate((B1_vector, B2_vector, B3_vector, B4_vector, B5_vector, B6_vector, B7_vector, B9_vector, B10_vector), axis=1)

In [15]:
bands_matrix.shape

(58304271, 9)

In [16]:
bands_matrix

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [17]:
band_factors = extract_factors(bands_matrix,3)

Evaluation of results using Principal Component Analysis(PCA) as data reduction criteria
Explained variance using  3  components:  99.2879443947 %


In [18]:
band_factors.shape

(58304271, 3)

In [19]:
test_factors = extract_factors(bands_matrix.T,3)

Evaluation of results using Principal Component Analysis(PCA) as data reduction criteria
Explained variance using  3  components:  98.0128759641 %


In [20]:
test_factors.shape

(9, 3)

In [21]:
rebuilt_matrix = np.dot(band_factors,test_factors.T)

In [22]:
rebuilt_matrix.shape

(58304271, 9)

In [23]:
rebuilt_matrix

array([[ 332.69451001,  -35.63401405,   54.91823068, ...,  450.288704  ,
        -319.33464698, -775.77347596],
       [ 332.69451424,  -35.6340147 ,   54.91823121, ...,  450.28871005,
        -319.33465144, -775.77348457],
       [ 332.69451426,  -35.63401471,   54.91823123, ...,  450.2887101 ,
        -319.3346515 , -775.77348472],
       ..., 
       [ 332.69451414,  -35.6340147 ,   54.9182312 , ...,  450.28870995,
        -319.33465139, -775.77348437],
       [ 332.69451414,  -35.6340147 ,   54.9182312 , ...,  450.28870995,
        -319.33465139, -775.77348437],
       [ 332.69451414,  -35.6340147 ,   54.9182312 , ...,  450.28870995,
        -319.33465139, -775.77348437]])

In [24]:
df_original = pd.DataFrame(original_input_matrix)
df_calculated = pd.DataFrame(rebuilt_matrix)

In [25]:
print 'Cosine similarity between original, and rebuild images:'
for i in df_original.columns:
    value = pairwise.cosine_similarity(df_original[i],df_calculated[i])
    if i < 7:
        print 'Band',(i+1),': ', value
    else:
        print 'Band',(i+2),': ', value



Cosine similarity between original, and rebuild images:
Band 1 :  [[-0.52924785]]
Band 2 :  [[ 0.52532227]]
Band 3 :  [[-0.54002619]]
Band 4 :  [[-0.53923166]]
Band 5 :  [[ 0.50853795]]
Band 6 :  [[-0.53509485]]
Band 7 :  [[-0.5387424]]
Band 9 :  [[ 0.52819287]]
Band 10 :  [[ 0.53043943]]


In [26]:
# NDVI Calculation for the original data and the processed data.
B4_Original = df_original[3]
B5_Original = df_original[4]
B4_PCA = df_calculated[3]
B5_PCA = df_calculated[4]


In [27]:
# Normalized Difference Vegetation Index(NDVI) Calculations 
NDVI_Original = ndvi(B5_Original,B4_Original)
NDVI_PCA = ndvi(B5_PCA,B4_PCA)

#Treat NaN
NDVI_Original = NDVI_Original.fillna(1)
NDVI_PCA = NDVI_PCA.fillna(1)

In [28]:
print 'Cosine Similarity Between NDVI panels from Original Data, and Rebuild Data:'
print pairwise.cosine_similarity(NDVI_Original,NDVI_PCA)

Cosine Similarity Between NDVI panels from Original Data, and Rebuild Data:
[[ 1.]]


In [52]:
# Temperature Modeling Using Bands 9(Near Infrared)
# Initial Test, using original dataset for regression algorithm benchmark

NIR = df_original[8]
target = np.array(NIR)
factor_matrix = df_original.ix[:,0:7]
factor_matrix = np.array(factor_matrix)
print 'Factor Matrix Shape: ',factor_matrix.shape
print 'Target Shape: ',target.shape

Factor Matrix Shape:  (58304271, 8)
Target Shape:  (58304271,)


In [53]:
# Spliting on training and testing groups and then apply Ridge Regression with 10-fold crossvalidation 
# Because of data size, 90% of it would be used on the test partition
factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.90, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)

In [54]:
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

In [55]:
print 'Test samples',eval_test.shape

Test samples (52473844,)


In [56]:
print 'Training samples',target_train.shape

Training samples (5830427,)


In [57]:
#Cosine Similarity Among Prediction and Labels On Training
pairwise.cosine_similarity(eval_train,target_train)

array([[ 0.98733079]])

In [58]:
# MSE
mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

17527832.806576081

In [59]:
#Cosine Similarity Among Prediction and Labels On Testing
pairwise.cosine_similarity(eval_test,target_test)

array([[ 0.98734757]])

In [60]:
# MSE
mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

17500855.738326289

In [61]:
# Repeat regression, but using the PCA processed data

NIR = df_calculated[8]
target = np.array(NIR)
factor_matrix = df_calculated.ix[:,0:7]
factor_matrix = np.array(factor_matrix)
print 'Factor Matrix Shape: ',factor_matrix.shape
print 'Target Shape: ',target.shape


Factor Matrix Shape:  (58304271, 8)
Target Shape:  (58304271,)


In [62]:
#Spliting on training and testing groups and then apply Ridge Regression with 10-fold crossvalidation 
factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.90, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)

In [63]:
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

In [64]:
#Cosine Similarity Among Prediction and Labels On Training
pairwise.cosine_similarity(eval_train,target_train)

array([[ 1.]])

In [65]:
#MSE
mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

6.9076870208137537e-18

In [66]:
#Cosine Similarity Among Prediction and Labels On Testing
pairwise.cosine_similarity(eval_test,target_test)

array([[ 1.]])

In [67]:
# MSE
mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

6.9071540716417226e-18